# Setup

In [ ]:
from src.hardware import DMD, ALP4
import numpy as np
import ctypes as ct

In [ ]:
frame_numbers = [0, 2, 1, 2]

flut = ALP4.tFlutWrite(
            nOffset=ct.c_long(0),
            nSize=ct.c_long(len(frame_numbers)),
            FrameNumbers=(ct.c_ulong * 4096)(*frame_numbers),
        )

In [ ]:
dmd=DMD()
alp4 = dmd._alp4

In [ ]:
img_sequence = np.zeros((3, *dmd.shape), dtype=np.uint8)

img_sequence[0, :, :] = 255
img_sequence[2, :, :dmd.shape[1]//2] = 0

img_sequence = img_sequence.ravel()

In [ ]:
alp4.SeqAlloc(nbImg=img_sequence.shape[0], bitDepth=1)
alp4.SeqPut(imgData = img_sequence)
alp4.SetTiming(pictureTime = 200_000)

# Linear display

In [ ]:
alp4.Run()

print("DMD is running with the sequence. Press Enter to stop...")
input("Press Enter to stop...")

alp4.Halt()

# FLUT display

In [ ]:
alp4.SeqControl(ALP4.ALP_FLUT_MODE, ALP4.ALP_FLUT_9BIT)
alp4.SeqControl(ALP4.ALP_FLUT_ENTRIES9, len(frame_numbers))
alp4.ProjControlEx(ALP4.ALP_FLUT_WRITE_9BIT, ct.byref(flut))

In [ ]:
alp4.Run()

print("DMD is running with the sequence. Press Enter to stop...")
input("Press Enter to stop...")

alp4.Halt()
alp4.FreeSeq()
alp4.Free()